# <font color=green> **AULA PRÁTICA 5**
-----

## Objetivo:
Mostrar a implementação do código do ALGORITMO DE DIJKSTRA para resolução do Problema de Caminho Mínimo


## Importando as bibliotecas da aula

In [1]:
import pandas as pd
import numpy as np
from math import radians
from sklearn.neighbors import DistanceMetric

### 1 - Criando o algortimo de Dijkstra

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Conceito

1. Escolhido um ponto (ou nó ou vértice) inicial, o método calcula o custo mínimo desde ponto para todos os demais da rede de transporte considerada.

2. O método parte de uma estimativa inicial para o custo mínimo (distância ou tempo) e vai ajustando.

3. Ele considera que um ponto está “fechado” quando já estiver sido obtido um caminho de custo mínimo do ponto inicial da busca até o ponto em análise.

 **ALGORITMO DIJKSTRA**



<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1_YoIFcyHzpybHPV-4ue2hMXBl8VhftHh' />
<figcaption>Algoritmo Dijkstra</figcaption></center>
</figure>


Exemplo



<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1uKAE9AgarSqQMa7vPi7Q3SvTgsMJlLIM' />
<figcaption>Exemplo caminho mínimo </figcaption></center>
</figure>




Para iniciar vamos utilizar uma matriz de distância corresponde ao grafo apresentado.

In [29]:
# PASSO 1 - TABELA DE ENTRADA
df1 = pd.read_excel('/content/drive/Shareddrives/Disciplina Modelagem/Exemplo1_dijkstra.xlsx',index_col=0)

#### Algoritmo de Dijkstra
Inicializando o Algoritmo


In [4]:
# Entradas:
no_inicial = 1       # Indica em qual nó vamos iniciar o nosso algoritmo (considerar o nome do Index)

d1 = df1.copy()       # copiar o dataframe para realizar as alterações

Definição dos conjuntos

In [5]:
R = [no_inicial]             # R: conjunto dos nós rotulados - lista com o nó inicial
NR = list(d1.index.values)   # NR: conjunto de nós não rotulados
NR.remove(no_inicial)
NR

print(f'conjunto nós rotulados R: {R}')
print(f'conjunto nós não rotulados NR: {NR}')

conjunto nós rotulados R: [1]
conjunto nós não rotulados NR: [2, 3, 4, 5, 6]


Criando a lista de distâncias do no_inicial até $i$ e a lista de predecessor do nó $i$

In [6]:
# d: lista de distância do no_inicial até i
d = [0] + len(NR)*[np.inf] 

# p: lista de predecessor do nó i 
p = [0] + (len(NR))*[len(d)+1]

# criação do dataframe:
resultados_parciais = pd.DataFrame({'distancia_inicial_i': d,'precessor_i': p},index = d1.index).T

ite = 0
print('')
print('===================================================')
print(f'iteração: {ite}')
print('')
print(resultados_parciais)


iteração: 0

                       1    2    3    4    5    6
distancia_inicial_i  0.0  inf  inf  inf  inf  inf
precessor_i          0.0  7.0  7.0  7.0  7.0  7.0


Iteração 0 - Como sair do primeiro nó para os seguintes?

In [7]:
# qual primeiro nó analisado?
k = R[-1]      # no inicial  - sempre o último elemento da lista de nós rotulados R
print(f'Primeiro nó analisado: {k}')

Primeiro nó analisado: 1


Para saírmos do nó inicial, devemor perguntar:

A distância atual é maior que a soma que a d(ultimo) mais o custo?

In [8]:
# Para saírmos do nó inicial e irmos para o outro, precisamos verificar se a distância atual é maior que d(ultimo) + custo. Se for, assume menor valor
for i in NR:        
    if (resultados_parciais.loc['distancia_inicial_i',i]) > resultados_parciais.loc['distancia_inicial_i',k] + d1.loc[k,i]:
        resultados_parciais.loc['distancia_inicial_i',i] = resultados_parciais.loc['distancia_inicial_i',k] + d1.loc[k,i]
        resultados_parciais.loc['precessor_i',i] = k
resultados_parciais

,1,2,3,4,5,6
distancia_inicial_i,0.0,5.0,4.0,10.0,inf,inf
precessor_i,0.0,1.0,1.0,1.0,7.0,7.0


In [9]:
# Testar duas condições: se a distância do ponto k para todos os outros pontos for infinito, o grafo é desconexo, caso contrário, o valor de k é o NÓ com a menor distância_inicial_i
if sum(resultados_parciais.loc['distancia_inicial_i',NR] == np.inf) == len(NR):
    print('grafo é desconexo e não existe caminho entre os vértices em R e aqueles NR')
else:
    k = [i for i in resultados_parciais.sort_values(axis=1,by = 'distancia_inicial_i').columns if i in NR][0]  # colocar na ordem crescente as colunas do dataframe, checar quais estão em  NR e pegar menor valor
    
    # removemos K dos nós não rotulados, pois agora ele é rotulado
    NR.remove(k)    
    # adicionamos k em R (lista dos nós rotulados)
    R.append(k)
    print(resultados_parciais)
    print('')
    print(f'Nó escolhido: {k}')
    print('')


                       1    2    3     4    5    6
distancia_inicial_i  0.0  5.0  4.0  10.0  inf  inf
precessor_i          0.0  1.0  1.0   1.0  7.0  7.0

Nó escolhido: 3



Qual a condição de parada? A lista NR (não rotulados) tem que estar vazia, ou o problema de ser gráfico desconexo, que aí para o processo. Como montar isso?


In [10]:
# Entradas:
no_inicial = 1  # Nó inicial (considerar o nome do index)

d1 = df1.copy()              # copiar o dataframe para fazer as alterações

# definição dos conjuntos

R = [no_inicial]             # R: conjunto dos nós rotulados - lista com o nó inicial
NR = list(d1.index.values)   # NR: conjunto de nós não rotulados
NR.remove(no_inicial)

# criando a lista de distâncias do no_inicial até i e a lista de predecessor do nó i

# d: lista de distância do no_inicial até i
d = [0] + len(NR)*[np.inf] 

# p: lista de predecessor do nó i 
p = [0] + (len(NR))*[len(d)+1]

# criação do dataframe:
resultados_parciais = pd.DataFrame({'distancia_inicial_i': d,'precessor_i': p},index = d1.index).T

ite = 0
print('')
print('===================================================')
print(f'iteração: {ite}')
print('')
print(resultados_parciais)

# Iteração 0 - Como sair do primeiro nó para os seguintes?

# qual primeiro nó analisado?

k = R[-1]      # no inicial  - sempre o último elemento da lista de nós rotulados R

while len(NR) != 0:  # quando a lista NR está vazia, o processo para
    ite += 1   # para sabermos em qual iteração estamos
    
    print('')
    print('===================================================')
    print(f'iteração: {ite}')
    print(f'Distância de {k} para i:')

    for i in NR:    # para cada valor de i em NR      
        
        # Para saírmos do nó inicial e irmos para o outro, precisamos verificar se a distância atual é maior que d(ultimo) + custo. Se for, assume menor valor
        if (resultados_parciais.loc['distancia_inicial_i',i]) > resultados_parciais.loc['distancia_inicial_i',k] + d1.loc[k,i]: 
            resultados_parciais.loc['distancia_inicial_i',i] = resultados_parciais.loc['distancia_inicial_i',k] + d1.loc[k,i]
            resultados_parciais.loc['precessor_i',i] = k
    
    # quais as condições possíveis:

    # grafo desconexo - notar que vai parar o processo (com o break)
    if sum(resultados_parciais.loc['distancia_inicial_i',NR] == np.inf) == len(NR):   # se a soma das distancias parciais do conjunto NR forem todas infinitas 
        print('grafo é desconexo e não existe caminho entre os vértices em R e aqueles NR')
        break
    else: # Caso contrário (ocorra o que esperamos), vamos atualizar o valor de k 
        k = [i for i in resultados_parciais.sort_values(axis=1,by = 'distancia_inicial_i').columns if i in NR][0]  # colocar na ordem crescente as colunas do dataframe, checar quais estão em  NR e pegar menor valor
        NR.remove(k)
        R.append(k)
        print(resultados_parciais)
        print('')
        print(f'Nó escolhido: {k}')
        print('')  


iteração: 0

                       1    2    3    4    5    6
distancia_inicial_i  0.0  inf  inf  inf  inf  inf
precessor_i          0.0  7.0  7.0  7.0  7.0  7.0

iteração: 1
Distância de 1 para i:
                       1    2    3     4    5    6
distancia_inicial_i  0.0  5.0  4.0  10.0  inf  inf
precessor_i          0.0  1.0  1.0   1.0  7.0  7.0

Nó escolhido: 3


iteração: 2
Distância de 3 para i:
                       1    2    3     4    5    6
distancia_inicial_i  0.0  5.0  4.0  10.0  9.0  inf
precessor_i          0.0  1.0  1.0   1.0  3.0  7.0

Nó escolhido: 2


iteração: 3
Distância de 2 para i:
                       1    2    3    4    5    6
distancia_inicial_i  0.0  5.0  4.0  7.0  7.0  inf
precessor_i          0.0  1.0  1.0  2.0  2.0  7.0

Nó escolhido: 4


iteração: 4
Distância de 4 para i:
                       1    2    3    4    5     6
distancia_inicial_i  0.0  5.0  4.0  7.0  7.0  10.0
precessor_i          0.0  1.0  1.0  2.0  2.0   4.0

Nó escolhido: 5


iteração: 

In [11]:
d1

,1,2,3,4,5,6
1,0.0,5.0,4.0,10.0,inf,inf
2,5.0,0.0,2.0,2.0,2.0,inf
3,4.0,2.0,0.0,inf,5.0,inf
4,10.0,2.0,inf,0.0,1.0,3.0
5,inf,2.0,5.0,1.0,0.0,2.0
6,inf,inf,inf,3.0,2.0,0.0


In [12]:
inicio = d1.index[0]
for final in d1.index:  
  caminho = str(final)
  while final != inicio:
      distancia_total = resultados_parciais.loc['distancia_inicial_i',max(d1.index.values)]
      caminho = str(resultados_parciais.loc['precessor_i',final]) + ' ---> ' + caminho    
      final = resultados_parciais.loc['precessor_i',final]
  print(caminho)
print(f'distância total: {distancia_total}')

1
1.0 ---> 2
1.0 ---> 3
1.0 ---> 2.0 ---> 4
1.0 ---> 2.0 ---> 5
1.0 ---> 2.0 ---> 5.0 ---> 6
distância total: 9.0


### 2 - Voltando ao Problema


##### Importando os dados

In [13]:
nome_arquivo = 'cvrp_RJ0_toy1_instancia_cluster4'
df = pd.read_csv('/content/drive/Shareddrives/Disciplina Modelagem/cvrp_RJ0_toy1_instancia_cluster3',index_col=0)

In [14]:
df.head(3)

,rotas,demandas,latitude,longitude,Cluster
0,origem,NaN,-22.747450,-42.881871,3
1,"{'lng': -42.92705867402125, 'lat': -22.7881124...",7.0,-22.788112,-42.927059,3
2,"{'lng': -42.827461886756275, 'lat': -22.725373...",10.0,-22.725374,-42.827462,3


#### 2.2 - Criando a Matriz de Distância

In [15]:
# Latitude e Longitude em radianos:
df['lat_rad'] = df['latitude'].apply(radians)
df['long_rad'] = df['longitude'].apply(radians)

In [16]:
# importando o pacote de distância e escolhendo a distância (usando o método de haversine)
dist = DistanceMetric.get_metric('haversine')

In [ ]:
# precisamos transformar para array
df[['lat_rad','long_rad']].to_numpy()

In [ ]:
# distância haversine - multiplicar por 6373 para passar para Km
dist.pairwise(df[['lat_rad','long_rad']].to_numpy())*6373

In [ ]:
## Finalmente
distancia1 = pd.DataFrame(dist.pairwise(df[['lat_rad','long_rad']].to_numpy())*6373)
distancia1

In [ ]:
# Entradas:
no_inicial = 0  # Nó inicial (considerar o nome do index)

d1 = distancia1.copy()              # copiar o dataframe para fazer as alterações

# definição dos conjuntos

R = [no_inicial]             # R: conjunto dos nós rotulados - lista com o nó inicial
NR = list(d1.index.values)   # NR: conjunto de nós não rotulados
NR.remove(no_inicial)

# criando a lista de distâncias do no_inicial até i e a lista de predecessor do nó i

# d: lista de distância do no_inicial até i
d = [0] + len(NR)*[np.inf] 

# p: lista de predecessor do nó i 
p = [0] + (len(NR))*[len(d)+1]

# criação do dataframe:
resultados_parciais = pd.DataFrame({'distancia_inicial_i': d,'precessor_i': p},index = d1.index).T

ite = 0
print('')
print('===================================================')
print(f'iteração: {ite}')
print('')
print(resultados_parciais)

# Iteração 0 - Como sair do primeiro nó para os seguintes?

# qual primeiro nó analisado?

k = R[-1]      # no inicial  - sempre o último elemento da lista de nós rotulados R

while len(NR) != 0:  # quando a lista NR está vazia, o processo para
    ite += 1   # para sabermos em qual iteração estamos
    
    print('')
    print('===================================================')
    print(f'iteração: {ite}')
    print(f'Distância de {k} para i:')

    for i in NR:    # para cada valor de i em NR      
        
        # Para saírmos do nó inicial e irmos para o outro, precisamos verificar se a distância atual é maior que d(ultimo) + custo. Se for, assume menor valor
        if (resultados_parciais.loc['distancia_inicial_i',i]) > resultados_parciais.loc['distancia_inicial_i',k] + d1.loc[k,i]: 
            resultados_parciais.loc['distancia_inicial_i',i] = resultados_parciais.loc['distancia_inicial_i',k] + d1.loc[k,i]
            resultados_parciais.loc['precessor_i',i] = k
    
    # quais as condições possíveis:

    # grafo desconexo - notar que vai parar o processo (com o break)
    if sum(resultados_parciais.loc['distancia_inicial_i',NR] == np.inf) == len(NR):   # se a soma das distancias parciais do conjunto NR forem todas infinitas 
        print('grafo é desconexo e não existe caminho entre os vértices em R e aqueles NR')
        break
    else: # Caso contrário (ocorra o que esperamos), vamos atualizar o valor de k 
        k = [i for i in resultados_parciais.sort_values(axis=1,by = 'distancia_inicial_i').columns if i in NR][0]  # colocar na ordem crescente as colunas do dataframe, checar quais estão em  NR e pegar menor valor
        NR.remove(k)
        R.append(k)
        print(resultados_parciais)
        print('')
        print(f'Nó escolhido: {k}')
        print('') 

In [21]:
resultados_parciais

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
distancia_inicial_i,0.0,6.475829,6.09786,4.915668,0.563399,5.605416,1.206249,7.530781,3.155748,3.717454,2.881976,0.762198,1.465667,3.173856,6.058339,4.141126,0.302652,0.158263,6.102317,3.231648,3.900783,7.19206,7.122735,0.688751,3.57533,5.094426,3.121243,3.332021,0.653043,0.554786,2.056627,2.911623,5.439083,6.439725,12.502276,0.155726,4.60322,4.285384
precessor_i,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [ ]:
inicio = d1.index[0]
for final in d1.index:  
  caminho = str(final)
  while final != inicio:
      distancia_total = resultados_parciais.loc['distancia_inicial_i',max(d1.index.values)]
      caminho = str(resultados_parciais.loc['precessor_i',final]) + ' ---> ' + caminho    
      final = resultados_parciais.loc['precessor_i',final]
  print(caminho)
print(f'distância total até o ultimo: {distancia_total}')

#### Caso 2

Criando uma nova Matriz de distancia com menos arcos

In [ ]:
# import random
# linha = []
# coluna = []
# for i in range(37):
#   n1 = random.randint(0,37)
#   n2= random.randint(0,37)
#   linha.append(n1)
#   coluna.append(n2)

In [ ]:
# import numpy as np
# for i in linha:
#   for j in coluna:
#     if i!=j:
#       d2.iloc[i,j] = np.inf

In [24]:
caso2 = pd.read_excel('/content/drive/Shareddrives/Disciplina Modelagem/Aulas/caso2.xlsx', index_col=0)

In [25]:
caso2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
0,0.000000,inf,6.097860,4.915668,inf,inf,1.206249,inf,inf,inf,2.881976,0.762198,inf,3.173856,6.058339,4.141126,inf,0.158263,6.102317,inf,inf,7.192060,inf,inf,inf,inf,inf,3.332021,inf,inf,inf,inf,5.439083,inf,12.502276,inf,4.603220,inf
1,6.475829,0.000000,12.371744,3.074381,6.185008,5.533697,7.553542,13.399600,8.733437,3.734635,9.123024,5.769134,7.496346,7.734448,6.421359,3.417173,6.686919,6.329205,2.319166,9.431924,10.235111,8.139268,3.106772,6.345820,9.225948,1.600747,9.037669,9.773549,5.873955,6.308741,8.473449,7.770894,3.387288,2.367190,18.941151,6.558024,10.486113,10.272777
2,inf,inf,0.000000,10.083234,inf,inf,4.907799,inf,inf,inf,3.260194,6.689610,inf,5.197635,11.388093,10.218937,inf,6.256086,12.200102,inf,inf,11.938881,inf,inf,inf,inf,inf,2.885178,inf,inf,inf,inf,11.484451,inf,6.703980,inf,2.208703,inf
3,inf,inf,10.083234,0.000000,inf,inf,5.676643,inf,inf,inf,6.978413,4.159185,inf,5.052889,7.713578,4.240569,inf,4.813322,4.621984,inf,inf,9.392311,inf,inf,inf,inf,inf,7.831498,inf,inf,inf,inf,5.094478,inf,16.778724,inf,8.012659,inf
4,0.563399,6.185008,6.561006,4.888189,0.000000,5.042778,1.724860,8.053101,3.718315,3.225774,3.386329,0.843769,2.026524,3.701356,5.497087,3.657937,0.863033,0.442988,5.654879,3.739701,4.362867,6.652655,6.645654,1.148840,4.134230,4.741422,3.669036,3.741322,0.376429,1.000180,2.509275,3.452090,4.935300,5.990878,12.877333,0.515905,5.140522,4.300900
5,inf,inf,11.211598,6.859537,inf,0.000000,6.718443,inf,inf,inf,8.291955,5.453275,inf,8.585692,0.894503,2.619740,inf,5.471984,3.355292,inf,inf,2.623931,inf,inf,inf,inf,inf,8.335252,inf,inf,inf,inf,2.152452,inf,16.863943,inf,10.113794,inf
6,1.206249,7.553542,4.907799,5.676643,1.724860,6.718443,0.000000,6.329913,2.116574,4.923628,1.676106,1.795932,0.556983,2.571300,7.104988,5.347198,0.922590,1.363423,7.298715,2.025401,2.718044,8.104660,8.327433,1.223257,2.476148,6.236039,1.965419,2.225116,1.855843,1.245339,0.924456,2.241422,6.644368,7.636382,11.388149,1.210104,3.418089,3.918620
7,7.530781,13.399600,2.132531,10.800074,8.053101,12.941331,6.329913,0.000000,4.674043,11.241467,4.674801,7.995872,6.138708,5.747591,13.202374,11.656883,7.235923,7.685592,13.546694,4.319647,3.875753,13.882023,14.616908,7.323059,4.181259,12.316577,4.462760,4.680857,8.170074,7.415420,5.613641,5.652118,12.967022,13.883442,6.661560,7.539678,2.960273,7.235220
8,3.155748,8.733437,3.846512,6.262737,3.718315,8.761069,2.116574,4.674043,0.000000,6.728307,1.358169,3.443659,1.693892,1.405668,9.193137,7.120234,2.855352,3.291674,8.928270,1.347008,2.172837,10.221234,10.027794,2.769175,0.492984,7.650128,0.618826,2.397213,3.727190,2.886851,1.976179,1.145695,8.450062,9.262674,10.549623,3.217597,1.762230,5.268810
9,inf,inf,9.785828,4.244811,inf,inf,4.923628,inf,inf,0.000000,6.598600,3.284935,inf,6.267135,3.498453,0.442232,inf,3.560754,2.493219,inf,inf,5.151130,inf,inf,inf,inf,inf,6.946800,inf,inf,inf,inf,1.727382,inf,16.006099,inf,8.298067,inf


In [26]:
# Entradas:
no_inicial = 0  # Nó inicial (considerar o nome do index)

d1 = caso2.copy()              # copiar o dataframe para fazer as alterações

# definição dos conjuntos

R = [no_inicial]             # R: conjunto dos nós rotulados - lista com o nó inicial
NR = list(d1.index.values)   # NR: conjunto de nós não rotulados
NR.remove(no_inicial)

# criando a lista de distâncias do no_inicial até i e a lista de predecessor do nó i

# d: lista de distância do no_inicial até i
d = [0] + len(NR)*[np.inf] 

# p: lista de predecessor do nó i 
p = [0] + (len(NR))*[len(d)+1]

# criação do dataframe:
resultados_parciais = pd.DataFrame({'distancia_inicial_i': d,'precessor_i': p},index = d1.index).T

ite = 0
print('')
print('===================================================')
print(f'iteração: {ite}')
print('')
print(resultados_parciais)

# Iteração 0 - Como sair do primeiro nó para os seguintes?

# qual primeiro nó analisado?

k = R[-1]      # no inicial  - sempre o último elemento da lista de nós rotulados R

while len(NR) != 0:  # quando a lista NR está vazia, o processo para
    ite += 1   # para sabermos em qual iteração estamos
    
    print('')
    print('===================================================')
    print(f'iteração: {ite}')
    print(f'Distância de {k} para i:')

    for i in NR:    # para cada valor de i em NR      
        
        # Para saírmos do nó inicial e irmos para o outro, precisamos verificar se a distância atual é maior que d(ultimo) + custo. Se for, assume menor valor
        if (resultados_parciais.loc['distancia_inicial_i',i]) > resultados_parciais.loc['distancia_inicial_i',k] + d1.loc[k,i]: 
            resultados_parciais.loc['distancia_inicial_i',i] = resultados_parciais.loc['distancia_inicial_i',k] + d1.loc[k,i]
            resultados_parciais.loc['precessor_i',i] = k
    
    # quais as condições possíveis:

    # grafo desconexo - notar que vai parar o processo (com o break)
    if sum(resultados_parciais.loc['distancia_inicial_i',NR] == np.inf) == len(NR):   # se a soma das distancias parciais do conjunto NR forem todas infinitas 
        print('grafo é desconexo e não existe caminho entre os vértices em R e aqueles NR')
        break
    else: # Caso contrário (ocorra o que esperamos), vamos atualizar o valor de k 
        k = [i for i in resultados_parciais.sort_values(axis=1,by = 'distancia_inicial_i').columns if i in NR][0]  # colocar na ordem crescente as colunas do dataframe, checar quais estão em  NR e pegar menor valor
        NR.remove(k)
        R.append(k)
        print(resultados_parciais)
        print('')
        print(f'Nó escolhido: {k}')
        print('') 



iteração: 0

                      0     1     2     3     4   ...    33    34    35    36    37
distancia_inicial_i  0.0   inf   inf   inf   inf  ...   inf   inf   inf   inf   inf
precessor_i          0.0  39.0  39.0  39.0  39.0  ...  39.0  39.0  39.0  39.0  39.0

[2 rows x 38 columns]

iteração: 1
Distância de 0 para i:
                      0     1        2   ...    35       36    37
distancia_inicial_i  0.0   inf  6.09786  ...   inf  4.60322   inf
precessor_i          0.0  39.0  0.00000  ...  39.0  0.00000  39.0

[2 rows x 38 columns]

Nó escolhido: 17


iteração: 2
Distância de 17 para i:
                      0          1        2   ...         35       36         37
distancia_inicial_i  0.0   6.487468  6.09786  ...   0.397791  4.60322   4.531248
precessor_i          0.0  17.000000  0.00000  ...  17.000000  0.00000  17.000000

[2 rows x 38 columns]

Nó escolhido: 35


iteração: 3
Distância de 35 para i:
                      0          1        2   ...         35       36       

In [30]:
inicio = d1.index[0]
for final in d1.index:  
  caminho = str(final)
  while final != inicio:
      distancia_total = resultados_parciais.loc['distancia_inicial_i',max(d1.index.values)]
      caminho = str(resultados_parciais.loc['precessor_i',final]) + ' ---> ' + caminho    
      final = resultados_parciais.loc['precessor_i',final]
  print(caminho)
print(f'distância total do último caminho: {distancia_total}')

0
0.0 ---> 17.0 ---> 1
0.0 ---> 2
0.0 ---> 3
0.0 ---> 17.0 ---> 4
0.0 ---> 17.0 ---> 5
0.0 ---> 6
0.0 ---> 6.0 ---> 7
0.0 ---> 6.0 ---> 8
0.0 ---> 17.0 ---> 9
0.0 ---> 10
0.0 ---> 11
0.0 ---> 6.0 ---> 12
0.0 ---> 13
0.0 ---> 14
0.0 ---> 15
0.0 ---> 17.0 ---> 16
0.0 ---> 17
0.0 ---> 18
0.0 ---> 6.0 ---> 19
0.0 ---> 6.0 ---> 20
0.0 ---> 21
0.0 ---> 17.0 ---> 22
0.0 ---> 17.0 ---> 23
0.0 ---> 6.0 ---> 24
0.0 ---> 17.0 ---> 25
0.0 ---> 6.0 ---> 26
0.0 ---> 27
0.0 ---> 17.0 ---> 28
0.0 ---> 17.0 ---> 29
0.0 ---> 6.0 ---> 30
0.0 ---> 17.0 ---> 31
0.0 ---> 32
0.0 ---> 17.0 ---> 33
0.0 ---> 34
0.0 ---> 17.0 ---> 35
0.0 ---> 36
0.0 ---> 17.0 ---> 37
distância total do último caminho: 4.531248247594514
